# Elliptic Curve Math - Examples with Libbitcoin System (C++)
In this example, we demonstrate the properties of scalar and elliptic curve operations over finite fields.
<br>
<img src="images/ec_math_operations.jpg" alt="drawing" style="" width="700px"/>

### Libbitcoin-System (C++) Setup

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System
#include <bitcoin/bitcoin.hpp> 

<hr style="border: 0.5px dashed #000;">


# 1 Scalar Operations


## 1.1 Generate a valid `secp256k1` scalar

Libbitcoin Classes/Methods:
* [`bc::base16_literal`](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/formats/base_16.hpp#L38)
* [`bc::to_chunk`](https://github.com/libbitcoin/libbitcoin/blob/master/include/bitcoin/bitcoin/utility/data.hpp#L120)
* [`bc::wallet::ec_private`](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/wallet/ec_private.hpp#L45)

In [2]:
// Create 128bits of entropy.
bc::data_chunk my_entropy(16); // 16 bytes = 128 bits
bc::pseudo_random_fill(my_entropy);

// Print out entropy.
std::cout << bc::encode_base16(my_entropy) << std::endl;


a1f0cbb35f7d0a609e6787e7c623bff9


In [3]:
// Create valid secp256k1 scalar from entropy.
auto my_prev_entropy = bc::to_chunk(bc::base16_literal("394e6ed80d23574456f4641f552a9eb7"));
bc::wallet::ec_private my_key(my_prev_entropy);

// Print out scalar.
std::cout << bc::encode_base16(my_key.secret()) << std::endl;


6996e113742aa9672e679a16bf70ddcfce0db2686a118f07526dd9ec1b61f058


## 1.2 Demonstrate Distributivity

**`(a + b) * c = a * c + b * c`**
<br>

* `a = 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b`
* `b = b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2`
* `c = 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf`

Libbitcoin Classes/Methods:
* [**`bc::ec_secret`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/math/elliptic_curve.hpp#L34)
* [**`bc::ec_add`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/math/elliptic_curve.hpp#L94) `(scalar += scalar)`
* [**`bc::ec_multiply`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/math/elliptic_curve.hpp#L106) `(scalar *= scalar)`

In [4]:
auto scalar_a = bc::base16_literal("86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b");
auto scalar_b = bc::base16_literal("b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2");
auto scalar_c = bc::base16_literal("6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf");

// Left side: (a + b) * c
bc::ec_secret scalar_result(scalar_a);
bc::ec_add(scalar_result, scalar_b);    // a += b
bc::ec_multiply(scalar_result, scalar_c);  // (a + b) *= c

// Right side: a * c + b * c
bc::ec_secret scalar_a_mult_c(scalar_a);
bc::ec_secret scalar_b_mult_c(scalar_b);
bc::ec_multiply(scalar_a_mult_c, scalar_c); // a *= c
bc::ec_multiply(scalar_b_mult_c, scalar_c); // a *= c
bc::ec_secret scalar_result_(scalar_a_mult_c);
bc::ec_add(scalar_result_, scalar_b_mult_c); // (a * c) += (b * c)

// Check for equality.
std::cout << (scalar_result == scalar_result_) << std::endl;  

// Print out.
std::cout << bc::encode_base16(bc::to_chunk(scalar_result)) << std::endl;
std::cout << bc::encode_base16(bc::to_chunk(scalar_result_)) << std::endl;


1
5052a8db6fbe0325b2231b630b4a4ec42b9c8e0d7b00e62c5ed1569c85e2733f
5052a8db6fbe0325b2231b630b4a4ec42b9c8e0d7b00e62c5ed1569c85e2733f


<hr style="border: 0.5px dashed #000;">

# 2 Elliptic Curve Operations:



## 2.1 Generate a valid `secp256k1` point

Libbitcoin-system Classes/Methods:
* [**`bc::wallet::ec_private::to_public()`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/wallet/ec_private.hpp#L114)
* [**`bc::wallet::ec_public::point()`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/wallet/ec_public.hpp#L71)
* [**`bc::ec_compressed`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/math/elliptic_curve.hpp#L39)

In [5]:
bc::data_chunk my_entropy_(16); // 16 bytes = 128 bits
bc::pseudo_random_fill(my_entropy_);
bc::wallet::ec_private my_key_(my_entropy_);
auto my_point = my_key_.to_public().point(); // Derive point

std::cout << bc::encode_base16(my_point) << std::endl;


02527d003fd7af2e81bb5d8060233f02bae82e99e3e68dc4d6f44fe58e65615a24


## 2.2) Demonstrate Distributivity

**`(a + b) * G = A + B`**

* `a = 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b`
* `b = b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2`

Libbitcoin-system Classes/Methods:

* [**`bc::secret_to_public()`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/math/elliptic_curve.hpp#L118)
* [**`bc::ec_add`**](https://github.com/libbitcoin/libbitcoin/blob/version3/include/bitcoin/bitcoin/math/elliptic_curve.hpp#L86) `(point += scalar * G)`

In [6]:
// Left side: (a + b) * G 
bc::ec_secret scalar_a_plus_b(scalar_a);
bc::ec_add(scalar_a_plus_b, scalar_b); 
bc::ec_compressed point;
bc::secret_to_public(point, scalar_a_plus_b);

// Right side: = A + B
bc::ec_compressed point_;
bc::secret_to_public(point_, scalar_a); 
bc::ec_add(point_, scalar_b); // point = point + b*G

// Check for equality.
std::cout << (point == point_) << std::endl;

// Print out points.
std::cout << bc::encode_base16(bc::to_chunk(point)) << std::endl;
std::cout << bc::encode_base16(bc::to_chunk(point_)) << std::endl;


1
023cbf38233fd17a7c3126e14e85a03c27c81b97f3158511c0ae79011dad88a60b
023cbf38233fd17a7c3126e14e85a03c27c81b97f3158511c0ae79011dad88a60b
